# <font color='darkblue'>Jane street data prediction</font>

#### This notebook includes a basic random forest classification model analysis. Each snippet of code shows explanation.Hopefully this will be helpful for someone who is trying kaggle competition for the first time.<br>

### <span style='background :yellow'>Please feel free to upvote and comment if you have any questions<span>

<h1><left>Lets get started!!</left></h1>

![](https://media1.tenor.com/images/7cb11ffe975452b5db7ca97b34b9d662/tenor.gif?itemid=19594590)

<font color='darkblue'>===================================================================================================================================================================
=================================================================================================================================================================<font>

### <center> <font color='darkblue'>                                 Lets import all necessary Libraries!<font>
### <center><font color='darkblue'>                            All packages that this notebook needs are below. :)<font>
    


![](https://media1.tenor.com/images/e5d4fff0602734f5e0d23ec814dc31c5/tenor.gif?itemid=19594862)

In [ ]:
import time
import statsmodels.api as sm
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import svm, datasets
from sklearn.tree import DecisionTreeClassifier
from tqdm.notebook import tqdm
import janestreet



### <center> <font color='darkblue'>                                 Import the train dataset<font>

#### Please note that I've uploaded only 10K rows. Please exclude nrow's in order to import the entire train dataset.

In [ ]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv',low_memory=False, nrows=10000)

### <center> <font color='darkblue'>                                 Create action variable<font>

1. Here we need to create an action variable. This can be done by using 'resp' variable. All the Resp column rows greater than 0 will be coded 1 for action column. And the rows with value less than or equal to 0 are coded as 0. This basically makes the Jane street analysis a binary classification.<br>
2. Independent variables 'resp_1','resp_2','resp_3','resp_4','resp','ts_id' are not showing up in test datasety. So, remove these from train dataset.


In [ ]:
#encode the numeric value to 1 and 0

train['action']=(train['resp'].values>0).astype(int)
f_mean=train.mean()
train=train.fillna(f_mean)

train=train.drop(['resp_1','resp_2','resp_3','resp_4','resp','ts_id'], axis=1)
train.shape

### <center> <font color='darkblue'>                                 Get features list.<font>

1. Trainx is a dataset without y variable. Now, as we need this for creating the model. This dataset basically has all the independent variable columns.
2. Features includes all the independent column names.

In [ ]:
# get the list of all x features to the list 
trainx=train.drop(['action'], axis=1)
cols=trainx.columns
features = cols.to_list()


### <center> <font color='darkblue'>                                Split the dataset for validation<font>

1. Split the dataset into train and validation data. 
2. I split the dataset to half. So, we have 5000 rows for train data and 5000 rows for validation. In other words, I will train the model on 5K rows on train data and test it on validation data to determine the accuracy.

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(trainx, train[['action']], test_size=0.5, random_state=0)

### <center> <font color='darkblue'>                                Creating the model and determining the ROC score from it.<font>

In [ ]:
colnameout=['Classifier','roc_auc']
approach=pd.DataFrame(columns=colnameout)


clf = RandomForestClassifier(n_estimators=200, random_state=0)
clf.fit(X_train, Y_train)

name='RandomForest'
pred_y_0 = clf.predict(X_val)
prob_y_valrf = clf.predict_proba(X_val)[:,1]
pred_y_0=pd.DataFrame(clf.predict(X_val))
pred_y_0.columns = ['Prob']

pred_y_0['ypred'] = np.where(pred_y_0['Prob']>=0.5,1,0)
pred_y_0=np.array(pred_y_0['ypred'] )


y_probs = clf.predict(X_val)

rocscore=roc_auc_score(Y_val, prob_y_valrf)


def plot_roc_auc(Y_val, prob_y_valrf, lbl):
    plt.figure(figsize=(8,6))
    roc_auc = roc_auc_score(Y_val, y_probs)
    fpr, tpr, thresholds = roc_curve(Y_val, prob_y_valrf)
    plt.plot(fpr, tpr, label='Random Forest ROC(area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate',fontsize=18)
    plt.ylabel('True Positive Rate',fontsize=18)
    plt.title('Random Forest characteristic- Random Forest classifier',fontsize=20)
    plt.legend(loc="lower right")
   
    plt.show()
   
plot_roc_auc(Y_val, prob_y_valrf, 'Random Forest ')    

### <center> <font color='darkblue'>                                Submission<font>

In [ ]:
env = janestreet.make_env()
env_iter = env.iter_test()

for (test_df, prediction_df) in env_iter:
    if test_df['weight'].item() == 0:
        prediction = 0
        prediction_df.action=prediction
        env.predict(prediction_df)
    else:
        test=test_df.fillna(f_mean)
        X_test = test.loc[:,features]
        y_preds=clf.predict(X_test)
        prediction_df.action=y_preds
        env.predict(prediction_df)